# **코스닥 시가총액 상위 종목 수집**

In [2]:
!git clone "https://github.com/FinanceData/marcap.git" marcap

Cloning into 'marcap'...
remote: Enumerating objects: 3218, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 3218 (delta 60), reused 84 (delta 60), pack-reused 3134
Receiving objects: 100% (3218/3218), 2.62 GiB | 25.18 MiB/s, done.
Resolving deltas: 100% (2250/2250), done.
Updating files: 100% (41/41), done.


In [55]:
!pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

from marcap import marcap_data
from datetime import datetime, timedelta

import FinanceDataReader as fdr


In [36]:
def get_date() :
    return (datetime.today() - timedelta(1)).strftime('%Y-%m-%d')

def get_kosdaq_top100cap_info() :
    str_yesterday = get_date()      # 어제 일자 출력 (추후 오늘로 수정)
    
    # Market Cap 데이터 수집
    df_market_cap = marcap_data(str_yesterday)

    # KOSDAQ 시총 100위 회사 출력
    df_kosdaq = df_market_cap[df_market_cap['MarketId'] == 'KSQ']
    df_kosdaq.sort_values(by = 'Marcap', ascending=False, inplace=True)
    df_kosdaq.reset_index(drop=True, inplace=True)
    ksq_cap_100 = df_kosdaq[:100][['Code', 'Name']].set_index('Code').to_dict()

    return ksq_cap_100['Name']

In [37]:
ksq_cap_100 = get_kosdaq_top100cap_info()

<ipython-input-36-80d300ce2163>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kosdaq.sort_values(by = 'Marcap', ascending=False, inplace=True)


In [ ]:
ksq_cap_100

# 종목별 주가 데이터 수집

In [88]:
def crawling_volume_info(ticker_info) :
    df_volume = None

    for ticker in tqdm(list(ticker_info.keys())) :
        end_date = (datetime.today() - timedelta(1)).strftime('%Y-%m-%d')
        start_date = '2015-01-01'

        if type(df_volume) == type(None) :
            df_volume = fdr.DataReader(ticker, start=start_date, end=end_date)
            df_volume = df_volume[['Volume']]
            df_volume.columns = [ticker]
        
        else :
            df_add = fdr.DataReader(ticker, start=start_date, end=end_date)
            df_add = df_add[['Volume']]
            df_add.columns = [ticker]
            df_volume = df_volume.merge(df_add, how = 'outer', left_index = True, right_index=True)
    
    return df_volume

100%|██████████| 100/100 [01:20<00:00,  1.25it/s]


In [90]:
df_volume = crawling_volume_info(ksq_cap_100)
df_volume.to_csv('volume_lst.csv')